# Parameter Management

Once we have chosen an architecture
and set our hyperparameters,
we proceed to the training loop,
where our goal is to find parameter values
that minimize our objective function.
After training, we will need these parameters
in order to make future predictions.
Additionally, we will sometimes wish
to extract the parameters
either to reuse them in some other context,
to save our model to disk so that
it may be executed in other software,
or for examination in the hopes of
gaining scientific understanding.

Most of the time, we will be able
to ignore the nitty-gritty details
of how parameters are declared
and manipulated, relying on the framework
to do the heavy lifting.
However, when we move away from
stacked architectures with standard layers,
we will sometimes need to get into the weeds
of declaring and manipulating parameters.
In this section, we cover the following:

* Accessing parameters for debugging, diagnostics, and visualizations.
* Parameter initialization.
* Sharing parameters across different model components.

We start by focusing on an MLP with one hidden layer.


In [1]:
import tensorflow as tf
import numpy as np

net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4, activation=tf.nn.relu),
    tf.keras.layers.Dense(1),
])

x = tf.random.uniform((2, 4))
net(x)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[-0.6632321],
       [-1.1436167]], dtype=float32)>

## Parameter Access

Let us start with how to access parameters
from the models that you already know.
When a model is defined via the `Sequential` class,
we can first access any layer by indexing
into the model as though it were a list.
Each layer's parameters are conveniently
located in its attribute.
We can inspect the parameters of the second fully-connected layer as follows.


In [2]:
print(net.layers[2].weights)

[<tf.Variable 'sequential/dense_1/kernel:0' shape=(4, 1) dtype=float32, numpy=
array([[ 0.302853 ],
       [ 0.5671247],
       [-0.9909644],
       [-1.0337962]], dtype=float32)>, <tf.Variable 'sequential/dense_1/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]


The output tells us a few important things.
First, this fully-connected layer
contains two parameters,
corresponding to that layer's
weights and biases, respectively.
Both are stored as single precision floats.
Note that the names of the parameters
allow us to *uniquely* identify
each layer's parameters,
even in a network containing hundreds of layers.


### Targeted Parameters

Note that each parameter is represented
as an instance of the parameter class.
To do anything useful with the parameters,
we first need to access the underlying numerical values.
There are several ways to do this.
Some are simpler while others are more general.
The following code extracts the bias
from the second neural network layer, which returns a parameter class instance, and
and further access that parameter's value.


In [3]:
print(type(net.layers[2].weights[1]))
print(net.layers[2].weights[1])
print(tf.convert_to_tensor(net.layers[2].weights[1]))

<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'>
<tf.Variable 'sequential/dense_1/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>
tf.Tensor([0.], shape=(1,), dtype=float32)


### All Parameters at Once

When we need to perform operations on all parameters,
accessing them one-by-one can grow tedious.
The situation can grow especially unwieldy
when we work with more complex blocks, (e.g., nested blocks),
since we would need to recurse
through the entire tree in to extract
each sub-block's parameters. Below we demonstrate accessing the parameters of the first fully-connected layer vs. accessing all layers.


In [4]:
print(net.layers[1].weights)
print(net.get_weights())

[<tf.Variable 'sequential/dense/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[ 0.38536662, -0.79716796,  0.7262034 ,  0.33552045],
       [-0.57293814,  0.56563157,  0.20567423,  0.4389941 ],
       [-0.5829506 ,  0.5240086 ,  0.7583799 ,  0.41485614],
       [-0.20305234, -0.8022177 ,  0.6321017 , -0.2647847 ]],
      dtype=float32)>, <tf.Variable 'sequential/dense/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]
[array([[ 0.38536662, -0.79716796,  0.7262034 ,  0.33552045],
       [-0.57293814,  0.56563157,  0.20567423,  0.4389941 ],
       [-0.5829506 ,  0.5240086 ,  0.7583799 ,  0.41485614],
       [-0.20305234, -0.8022177 ,  0.6321017 , -0.2647847 ]],
      dtype=float32), array([0., 0., 0., 0.], dtype=float32), array([[ 0.302853 ],
       [ 0.5671247],
       [-0.9909644],
       [-1.0337962]], dtype=float32), array([0.], dtype=float32)]


This provides us with another way of accessing the parameters of the network:


In [5]:
net.get_weights()[1]

array([0., 0., 0., 0.], dtype=float32)

### Collecting Parameters from Nested Blocks

Let us see how the parameter naming conventions work
if we nest multiple blocks inside each other.
For that we first define a function that produces blocks
(a block factory, so to speak) and then
combine these inside yet larger blocks.


In [6]:
def block1(name):
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(4, activation=tf.nn.relu)],
        name=name)

def block2():
    net = tf.keras.Sequential()
    for i in range(4):
        net.add(block1(name=f'block-{i}'))
    return net

rgnet = tf.keras.Sequential()
rgnet.add(block2())
rgnet.add(tf.keras.layers.Dense(1))
rgnet(x)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[-0.05686879],
       [-0.034129  ]], dtype=float32)>

Now that we have designed the network,
let us see how it is organized.


In [7]:
print(rgnet.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_2 (Sequential)    multiple                  80        
_________________________________________________________________
dense_6 (Dense)              multiple                  5         
Total params: 85
Trainable params: 85
Non-trainable params: 0
_________________________________________________________________
None


Since the layers are hierarchically nested,
we can also access them as though
indexing through nested lists.
For instance, we can access the first major block,
within it the second subblock,
and within that the bias of the first layer,
with as follows:


In [8]:
rgnet.layers[0].layers[1].layers[1].weights[1]

<tf.Variable 'sequential_1/sequential_2/block-1/dense_3/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>

## Parameter Initialization

Now that we know how to access the parameters,
let us look at how to initialize them properly.
We discussed the need for initialization in :numref:`sec_numerical_stability`.
The framework provides default random initializations to its layers.
However, we often want to initialize our weights
according to various other protocols. The framework provides most commonly
used protocols, and also allows to create a customer initializer.


By default, Keras initializes weight matrices uniformly by drawing from a range that is computed according to the input and output dimension, and the bias parameters are all set to $0$.
TensorFlow provides a variety of initialization methods both in the root module and the `keras.initializers` module.


### Built-in Initialization

Let us begin by calling on built-in initializers.
The code below initializes all weight parameters
as Gaussian random variables
with standard deviation $.01$, while bias parameters set to 0.


In [9]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4, activation=tf.nn.relu,
        kernel_initializer=tf.random_normal_initializer(mean=0, stddev=0.01),
        bias_initializer=tf.zeros_initializer()),
    tf.keras.layers.Dense(1)])

net(x)
net.weights[0], net.weights[1]

(<tf.Variable 'sequential_3/dense_7/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[-0.0057769 ,  0.00870675,  0.00765279,  0.00376331],
        [ 0.00248387,  0.00837455, -0.00295483,  0.01806357],
        [ 0.01814149, -0.00704269, -0.0123057 ,  0.01169605],
        [-0.00843834,  0.0039065 , -0.01451407, -0.0175827 ]],
       dtype=float32)>,
 <tf.Variable 'sequential_3/dense_7/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>)

We can also initialize all parameters
to a given constant value (say, $1$).


In [10]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4, activation=tf.nn.relu,
        kernel_initializer=tf.keras.initializers.Constant(1),
        bias_initializer=tf.zeros_initializer()),
    tf.keras.layers.Dense(1),
])

net(x)
net.weights[0], net.weights[1]

(<tf.Variable 'sequential_4/dense_9/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], dtype=float32)>,
 <tf.Variable 'sequential_4/dense_9/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>)

We can also apply different initializers for certain Blocks.
For example, below we initialize the first layer
with the `Xavier` initializer
and initialize the second layer
to a constant value of 42.


In [11]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4,
        activation=tf.nn.relu,
        kernel_initializer=tf.keras.initializers.GlorotUniform()),
    tf.keras.layers.Dense(
        1, kernel_initializer=tf.keras.initializers.Constant(1)),
])

net(x)
print(net.layers[1].weights[0])
print(net.layers[2].weights[0])

<tf.Variable 'sequential_5/dense_11/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[-0.8469932 ,  0.7953264 ,  0.64614505, -0.7553199 ],
       [-0.6462534 , -0.52710485,  0.19703287, -0.23398954],
       [-0.32029384,  0.02302605, -0.45530203, -0.75537956],
       [ 0.6050642 , -0.3986385 , -0.33967108,  0.8266712 ]],
      dtype=float32)>
<tf.Variable 'sequential_5/dense_12/kernel:0' shape=(4, 1) dtype=float32, numpy=
array([[1.],
       [1.],
       [1.],
       [1.]], dtype=float32)>


### Custom Initialization

Sometimes, the initialization methods we need
are not provided by the framework.
In the example below, we define an initializer
for the following strange distribution:

$$
\begin{aligned}
    w \sim \begin{cases}
        U[5, 10] & \text{ with probability } \frac{1}{4} \\
            0    & \text{ with probability } \frac{1}{2} \\
        U[-10, -5] & \text{ with probability } \frac{1}{4}
    \end{cases}
\end{aligned}
$$


Here we define a subclass of `Initializer` and implement the `__call__`
function that return a desired tensor given the shape and data type.


In [12]:
class MyInit(tf.keras.initializers.Initializer):
    def __call__(self, shape, dtype=None):
        return tf.random.uniform(shape, dtype=dtype)

net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4,
        activation=tf.nn.relu,
        kernel_initializer=MyInit()),
    tf.keras.layers.Dense(1),
])

net(x)
print(net.layers[1].weights[0])

<tf.Variable 'sequential_6/dense_13/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[0.9055351 , 0.8475363 , 0.27910316, 0.8872222 ],
       [0.37282085, 0.63385856, 0.77730024, 0.13232434],
       [0.03811538, 0.68917084, 0.41126502, 0.9393165 ],
       [0.009606  , 0.22611439, 0.5023135 , 0.44870007]], dtype=float32)>


Note that we always have the option
of setting parameters directly.


In [13]:
net.layers[1].weights[0][:].assign(net.layers[1].weights[0] + 1)
net.layers[1].weights[0][0, 0].assign(42)
net.layers[1].weights[0]

<tf.Variable 'sequential_6/dense_13/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[42.       ,  1.8475363,  1.2791032,  1.8872222],
       [ 1.3728209,  1.6338586,  1.7773002,  1.1323243],
       [ 1.0381154,  1.6891708,  1.411265 ,  1.9393165],
       [ 1.009606 ,  1.2261144,  1.5023135,  1.4487001]], dtype=float32)>

## Tied Parameters

Often, we want to share parameters across multiple layers.
Later we will see that when learning word embeddings,
it might be sensible to use the same parameters
both for encoding and decoding words.
We discussed one such case when we introduced :numref:`sec_model_construction`.
Let us see how to do this a bit more elegantly.
In the following we allocate a dense layer
and then use its parameters specifically
to set those of another layer.


In [14]:
# tf.keras behaves a bit differently. It removes the duplicate layer
# automatically
shared = tf.keras.layers.Dense(4, activation=tf.nn.relu)
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    shared,
    shared,
    tf.keras.layers.Dense(1),
])

net(x)
# Check whether the parameters are different
print(len(net.layers) == 3)

True


This example shows that the parameters
of the second and third layer are tied.
They are not just equal, they are
represented by the same exact tensor.
Thus, if we change one of the parameters,
the other one changes, too.
You might wonder,
*when parameters are tied
what happens to the gradients?*
Since the model parameters contain gradients,
the gradients of the second hidden layer
and the third hidden layer are added together
during backpropagation.

## Summary

* We have several ways to access, initialize, and tie model parameters.
* We can use custom initialization.


## Exercises

1. Use the FancyMLP defined in :numref:`sec_model_construction` and access the parameters of the various layers.
1. Look at the initialization module document to explore different initializers.
1. Construct a multilayer perceptron containing a shared parameter layer and train it. During the training process, observe the model parameters and gradients of each layer.
1. Why is sharing parameters a good idea?


[Discussions](https://discuss.d2l.ai/t/269)
